In [1]:
import numpy as np
import bet.sample as samp
import bet.sampling.basicSampling as bsam
import bet.calculateP.simpleFunP as simpleFunP
import bet.calculateP.calculateP as calculateP

from scipy.stats import uniform as uni

In [2]:
def make_model(skew_range):
    # this function makes a linear map whos first component is the x-unit vector
    # and each subsequent component is a norm-1 vector satisfying the property
    # that the 2-2 map made from it and the aforementioned unit vector is a map
    # with skewness in skew_range, which is a list of desired skewnesses   
    # TODO currently this map only works for 2-D input space     
    
    def my_model(parameter_samples):
        Q_map = [ [1.0, 0.0] ] # all map components have the same norm, rect_size to have measures of events equal btwn spaces.
        for s in skew_range:
            Q_map.append( [np.sqrt(s**2 - 1), 1] ) # taken with the first component, this leads to a 2-2 map with skewsness 's'
        Q_map = np.array( Q_map )
        QoI_samples = np.dot(parameter_samples, np.transpose(Q_map))
        return QoI_samples
    return my_model


def invert_rect_cb(My_Discretization, QoI_indices, Qref, rect_size, cells_per_dimension = 1):
    # Take full discretization objects, a set of indices for the QoI you want 
    # to use to perform inversion, and then do so by redefining the output spaces
    # based on these indices and solving the problem as usual.
    # My_Discretization is copied, the other two are pass-by-reference
    # Make copy of object so we can reference it before/after
    input_samples = My_Discretization._input_sample_set.copy()
    output_samples = My_Discretization._output_sample_set.copy()

    output_samples._dim = len(QoI_indices)
    output_samples.set_values(output_samples._values[:, QoI_indices])

    output_samples.global_to_local()

    my_discretization = samp.discretization(input_sample_set=input_samples,
                                            output_sample_set=output_samples)
    
    simpleFunP.regular_partition_uniform_distribution_rectangle_size(my_discretization, 
        Q_ref =  Qref[QoI_indices],
        rect_size = rect_size,
        cells_per_dimension = 1)
    
    calculateP.prob(my_discretization) # this updates the my_discretization object probabilities attribute
    
    return my_discretization


## Define Input Samples

In [3]:
num_samples = int(1E2) # this will be NUM SAMPLES PER DIM if you specify regular sampling.
dim_input = 2
dim_output = 2
dim_range_each = [0, 1] # same uncertainty bounds for each parameter
dim_range = np.repeat([dim_range_each], dim_input, axis=0)
Input_Samples = samp.sample_set(dim_input)

In [4]:
data_discretization_type = 'rand' # 'reg' for regular smapling, otherwise, random.

if data_discretization_type == 'reg': # regular mesh for partition
    Input_Samples = bsam.regular_sample_set(input_samples, 
                                            num_samples_per_dim = np.repeat(num_samples, dim_input, axis=0))
else: # random mesh for partition
    Input_Samples = bsam.random_sample_set('random', Input_Samples, num_samples = num_samples)
Input_Samples.estimate_volume_mc() # Utilize the MC assumption for volumes          

In [5]:
# Let's see what is in the probabilities vector ... it is empty.
Input_Samples._probabilities

## Define Map, Data Space

In [6]:
skew_range = [1, 2] # the range of skewness values you want to compare
QoI_choice_list = [ [0, n+1] for n in range(len(skew_range))] # this creates that range by subselecting indices from a special map we defined (see above)

my_model = make_model(skew_range)
sampler = bsam.sampler(my_model)


In [7]:
My_Discretization = sampler.compute_QoI_and_create_discretization(Input_Samples)

In [8]:
My_Discretization._output_sample_set._probabilities # notice the probabilities set is empty

## Define Observed Density

In [9]:
ref_input = 0.5*np.ones(dim_input)
Qref =  my_model(ref_input)
rect_area = 0.1 # the total volume measure of the observed density's support
rect_size = np.power(rect_area, 1./dim_output) # make box with this sidelength. it will have support with measure equal to `rect_area`


In [10]:
# see that probabilities are now defined.
My_Discretization._output_probability_set

In [11]:
map_number = 0
QoI_indices = QoI_choice_list[map_number]

In [12]:
# Make copy of object so we can reference it before/after
input_samples = My_Discretization._input_sample_set.copy()
output_samples = My_Discretization._output_sample_set.copy()

output_samples._dim = len(QoI_indices)
output_samples.set_values(output_samples._values[:, QoI_indices])

output_samples.global_to_local()

my_discretization = samp.discretization(input_sample_set=input_samples,
                                        output_sample_set=output_samples)

In [13]:
my_discretization._output_probability_set # look. it's empty.

In [14]:
# Compute the simple function approximation to the distribution on the data space of interest
simpleFunP.regular_partition_uniform_distribution_rectangle_size(my_discretization, 
        Q_ref =  Qref[QoI_indices],
        rect_size = rect_size,
        cells_per_dimension = 1)


In [15]:
my_discretization._output_probability_set # now there's something there.

In [16]:
my_discretization._output_probability_set._probabilities

array([ 1.,  0.])

In [17]:
my_discretization._output_probability_set._values # the samples in D used to define the density

array([[ 0.5,  0.5],
       [ inf,  inf]])

## Solve Inverse Problem

In [18]:
calculateP.prob(my_discretization) # this updates the my_discretization object probabilities attribute

In [19]:
my_discretization._input_sample_set._probabilities

array([ 0. ,  0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0.1,  0. ,  0.1,  0.1,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0.1,  0.1,  0. ,
        0. ,  0. ,  0. ,  0.1,  0. ,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0. ,
        0. ])

In [20]:
filename = 'test_saved_discretization'
samp.save_discretization(my_discretization, filename)

'test_saved_discretization'

## Visualize Solution

## Now for Consistent Bayes...

In [21]:
# This is the vector we'll be comparing against
vol_lambda = 1.0 # measure of the parameter space.
# set the prior
prior = 1.0/vol_lambda*np.ones(num_samples)

In [22]:
input_samples = my_discretization._input_sample_set._values # points from the parameter space

In [23]:
output_samples = my_discretization._output_sample_set._values # and where they map to.

In [24]:
from scipy.stats import uniform
from scipy.stats import gaussian_kde as gkde

In [25]:
data_kde = gkde(output_samples.transpose()) # pushforward of prior
pf_pr = data_kde.pdf(output_samples.transpose())

In [26]:
for m in QoI_indices:
    d = Qref[m] - 0.5*rect_size # marginal distribution ... left-hand side. (loc)
    obs = uni.pdf(output_samples[:,m].transpose(), loc=d, scale=rect_size)
    prior *= obs
prior /= pf_pr

In [27]:
post = prior/np.max(prior)

In [31]:
nonzero_cells = my_discretization._input_sample_set._probabilities/np.max(my_discretization._input_sample_set._probabilities)

In [33]:
# A check to see if the CB approach assigned positive probabilities where the BET assigned zeros.
for i in range(num_samples):
    if post[i] > nonzero_cells[i]:
        print "mismatch"